In [ ]:
import os
import torch
import json

import torch
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import open3d as o3d

path = "/mnt/wwn-0x5000000000000001-part1/datasets/EgoExo4d/gt_output_dir/annotation/auto/ego_pose_gt_anno_train_public.json"
count = 0
count_all = 0
data = json.load(open(path))
id_list = []
for id in data:
    if id not in id_list:
        id_list.append(id)
    right_landmarks = []
    left_landmarks = []
    len_right = 0
    len_left = 0
    len_all = 0
    for el in data[id]:
        landmark_3d_raw = data[id][el]
        right_hand_landmarks = np.array(landmark_3d_raw['right_hand_3d'])
        left_hand_landmarks = np.array(landmark_3d_raw['left_hand_3d'])
        if right_hand_landmarks.shape[0] == 0:
            right_hand_landmarks = np.ones((21, 3))*np.NaN
        if left_hand_landmarks.shape[0] == 0:
            left_hand_landmarks = np.ones((21, 3))*np.NaN
        right_landmarks.append(right_hand_landmarks)
        left_landmarks.append(left_hand_landmarks)
        if right_hand_landmarks.shape[0] >0:
            len_right  += 1
        if left_hand_landmarks.shape[0] >0:
            len_left += 1
        len_all += 1
    print(len_right, len_left, len_all)
    if len_right == len_left and len_right == len_all:
        count += 1
    count_all += 1
    if count == 1:
        break

print(count, count_all)


In [ ]:

# Create the initial 3D scatter plot
fig = go.Figure(
    data=[go.Scatter3d(
        x=right_landmarks[0][:, 0],
        y=right_landmarks[0][:, 1],
        z=right_landmarks[0][:, 2],
        mode='markers',
        marker=dict(size=2, color="blue", opacity=0.7)
    ),
    go.Scatter3d(
                x=left_landmarks[0][:, 0],
                y=left_landmarks[0][:, 1],
                z=left_landmarks[0][:, 2],
                mode='markers',
                marker=dict(size=2, color="red", opacity=0.7)
            )
    ],
    layout=go.Layout(
        title="Evolution of landmarks",
        updatemenus=[{
            "buttons": [
                {"args": [None, {"frame": {"duration": 500, "redraw": True}, "fromcurrent": True}],
                 "label": "Play",
                 "method": "animate"},
                {"args": [[None], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate"}],
                 "label": "Pause",
                 "method": "animate"}
            ],
            "direction": "left",
            "pad": {"r": 10, "t": 87},
            "showactive": False,
            "type": "buttons",
            "x": 0.1,
            "xanchor": "right",
            "y": 0,
            "yanchor": "top"
        }]
    ),
    frames=[
        go.Frame(
            data=[go.Scatter3d(
                x=left_landmarks[t][:, 0],
                y=left_landmarks[t][:, 1],
                z=left_landmarks[t][:, 2],
                mode='markers',
                marker=dict(size=2, color="red", opacity=0.7)
            ),
            go.Scatter3d(
                x=right_landmarks[t][:, 0],
                y=right_landmarks[t][:, 1],
                z=right_landmarks[t][:, 2],
                mode='markers',
                marker=dict(size=2, color="blue", opacity=0.7)
            )
            ],
            name=f"Frame {t}"
        ) for t in range(1, len(left_landmarks))
    ]
)

# Show the animated plot
fig.show()

In [ ]:
import json
path = "/mnt/wwn-0x5000000000000001-part1/datasets/EgoExo4d/gt_output_dir/annotation/auto/ego_pose_gt_anno_val_public.json"
tgt_dst = "/mnt/wwn-0x5000000000000001-part1/datasets/EgoExo4d/data_val/raw/"
data = json.load(open(path))

In [ ]:
i = 0
for id in data:
    dict = data[id]
    file_path = tgt_dst + "data_" + str(i) + ".json"
    with open(file_path, "w") as json_file:
        json.dump(dict, json_file, indent=4)
    i += 1

In [ ]:
import torch
import numpy as np
from tqdm import tqdm
import os
import json
import os.path as osp
from torch.utils.data import Dataset, TensorDataset
import controldiffeq
from torch.utils.data.dataloader import DataLoader

class Ego4D_dataloader():
    def __init__(self, path, batch_size, shuffle=True, path_data = None):
        self.path = path
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.path_data = path_data
        self.path_times = self.path.replace("loader", "times")
    
    def load_dataloader(self):
        loader = torch.load(self.path)
        times = torch.load(self.path_times)
        return loader, times
    
    def save_dataloader(self):
        print("Bulding the data...")
        dataset_landmarks = []
        dataset_task = []
        dataset_id = []
        for file in tqdm(os.listdir(self.path_data)):
            data = json.load(open(osp.join(self.path_data, file)))
            right_landmarks = []
            left_landmarks = []
            landmarks = []
            for el in data:
                landmark_3d_raw = data[el]
                right_hand_landmarks = np.array(landmark_3d_raw['right_hand_3d'])
                left_hand_landmarks = np.array(landmark_3d_raw['left_hand_3d'])
                if right_hand_landmarks.shape[0] == 0:
                    right_hand_landmarks = np.ones((21, 3))*np.NaN
                if left_hand_landmarks.shape[0] == 0:
                    left_hand_landmarks = np.ones((21, 3))*np.NaN
                landmarks.append(np.concatenate((right_hand_landmarks, left_hand_landmarks), axis=0))
                right_landmarks.append(right_hand_landmarks)
                left_landmarks.append(left_hand_landmarks)
                metadata = data[el]['metadata']
                take_uid = metadata['take_uid']
                task = metadata['take_name']
                task = get_task(task)
            landmarks = torch.tensor(landmarks)
            dataset_task.append(task)
            dataset_landmarks.append(landmarks)
            dataset_id.append(take_uid)
        dataset_land_padded, length = padding(dataset_landmarks)
        times = torch.arange(length).float()
        dataset_splineconv = SplineCoeff(dataset_land_padded, times)       
        data = Ego4D(*dataset_splineconv, torch.tensor(dataset_task), dataset_id)
        dataloader = DataLoader(data, batch_size=self.batch_size, shuffle=self.shuffle)
        torch.save(dataloader, self.path)
        torch.save(times, self.path_times)
        return dataloader, times

class Ego4D(Dataset):
    def __init__(self, data, labels, id, transform=None):
        self.data = data
        self.labels = labels
        self.id =  id
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        data = self.data[idx]
        label = self.labels[idx]
        id = self.id[idx]
        if self.transform:
            data= self.transform(data)
        return data, label, id
                
def padding(dataset):
    print("Padding...")
    length = 0
    for data in dataset:
        length = np.max([length, len(data)])
    dataset_padded = []
    for data in dataset:
        N, M, T = data.shape
        padded_tensor = torch.full((length, M, T), float('nan'))
        padded_tensor[:N, :, :] = data
        dataset_padded.append(padded_tensor)
    return torch.stack(dataset_padded), length

def get_task(task):
    if "basketball" in task:
        task = 0
    elif "dance" in task or "Dance" in task:
        task = 1
    elif "bike" in task:
        task = 2
    elif "music" in task:
        task = 3
    elif "covid" in task:
        task = 4
    elif "cooking" in task:
        task = 5
    elif "bouldering" in task:
        task = 6
    elif "Guitar" in task:
        task = 7
    elif "Violin" in task:
        task = 8
    elif "Piano" in task:
        task = 9
    elif "soccer" in task:
        task = 10
    elif "cpr" in task:
        task = 11 
    return task    

def SplineCoeff(dataset_landmarks, times):
    print("Calculating Spline Coeff...")
    dataset_splineconv = []
    land = dataset_landmarks.permute(0,2,1,3).float()
    dataset_splineconv = controldiffeq.natural_cubic_spline_coeffs(times, land)
    return dataset_splineconv



In [ ]:
from tqdm import tqdm
import os
import os.path as osp
import json
import numpy as np

path = "/mnt/wwn-0x5000000000000001-part1/datasets/EgoExo4d/data_train/raw"

time = []
for file in tqdm(os.listdir(path)):
    data = json.load(open(osp.join(path, file)))
    landmarks = []
    for el in data:
        landmark_3d_raw = data[el]
        right_hand_landmarks = np.array(landmark_3d_raw['right_hand_3d'])
        left_hand_landmarks = np.array(landmark_3d_raw['left_hand_3d'])
        if right_hand_landmarks.shape[0] == 0:
            right_hand_landmarks = np.ones((21, 3))*np.NaN
        if left_hand_landmarks.shape[0] == 0:
            left_hand_landmarks = np.ones((21, 3))*np.NaN
        landmarks.append(np.concatenate((right_hand_landmarks, left_hand_landmarks), axis=0))
    time.append(len(landmarks))



In [ ]:
import matplotlib.pyplot as plt

# Plotting
plt.figure(figsize=(8, 5))
plt.plot(np.sort(time)[:200], linestyle='-', color='b', label='Time')
plt.title("Length of samples")
plt.xlabel("Index")
plt.ylabel("Value")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
#create an array where only the time is > 100
time = np.array(time)
time_100 = time[time > 100]
time_sum = np.sum(time_100)
time_sum_samples = time_sum/100
time_sum_samples

In [2]:
import torch

path = torch.load("/home/studenti/agnelli/projects/4D_hands/basic_version/results/data_noNaN/train_loader.pt")
label = []
for data in path:
    label.append(data[4])


In [3]:
import numpy as np

# Example vector with n values between 0 and 11
vector = np.array(label)

# Count occurrences of each label
unique, counts = np.unique(vector, return_counts=True)

# Calculate percentages
percentages = {label: (count / len(vector)) * 100 for label, count in zip(unique, counts)}

# Display percentages
for label, percentage in percentages.items():
    print(f"Label {label}: {percentage:.2f}%")


Label 2: 16.91%
Label 3: 1.46%
Label 4: 43.76%
Label 5: 5.00%
Label 7: 0.44%
Label 8: 2.72%
Label 9: 29.07%
Label 11: 0.63%


In [ ]:
from dataloader import Ego4D_dataloader
import os
from torch.utils.data.dataloader import DataLoader
path = "/mnt/wwn-0x5000000000000001-part1/datasets/EgoExo4d/data_train/raw"
path_loader = "/home/studenti/agnelli/projects/4D_hands/basic_version/results/data/train_loader.pt"

batch_dim = 64
train_dataset = Ego4D_dataloader(path_loader, path_data = path)
loader = train_dataset.load_data()

In [ ]:
from torch.utils.data import Dataset, TensorDataset
dataset = filter_tensor_dataset(loader, 0.3)

In [53]:
train_loader = DataLoader(dataset, batch_size=64, shuffle=True)

In [8]:
import numpy as np

x = np.random.randn(5, 3)
t = x.size
t

15